In [3]:
# dependencies
import requests
import json
import pandas as pd
import time
import os

# import spotipy
#!pip install spotipy
import spotipy
import requests
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
from config import spotify_api as sp_client, spotify_client_secret as sp_secret

In [6]:
# set up client credentials 
# https://spotipy.readthedocs.io/en/master/?highlight=spotifyclientcredentials#client-credentials-flow
# API info: https://developer.spotify.com/documentation/web-api/reference/#/
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(sp_client, sp_secret))

In [12]:
# API info: https://developer.spotify.com/documentation/web-api/
## https://developer.spotify.com/documentation/web-api/reference/#/

In [8]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': sp_client,
    'client_secret': sp_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [9]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [10]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

In [11]:
r = r.json()
r

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [14]:
artist_id = '36QJpDe2go2KgaRleHCDTp'

# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()
d

{'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp/albums?offset=0&limit=50&include_groups=album',
 'items': [{'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36QJpDe2go2KgaRleHCDTp'},
     'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp',
     'id': '36QJpDe2go2KgaRleHCDTp',
     'name': 'Led Zeppelin',
     'type': 'artist',
     'uri': 'spotify:artist:36QJpDe2go2KgaRleHCDTp'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
    'CR',
    'CV',
    'CW',
    'CY',
    'CZ',
    'DE',
    'DJ',
    'DK',
    'DM',
    'DO',
    'DZ',
    'EC',
    'EE',

In [15]:
for album in d['items']:
    print(album['name'], ' --- ', album['release_date'])

The Complete BBC Sessions (Remastered)  ---  2016-09-16
Physical Graffiti (Deluxe Edition)  ---  2015-02-24
Celebration Day  ---  2012-11-19
Mothership (Remastered)  ---  2007
How the West Was Won (Remaster)  ---  2003-05-27
Coda (Deluxe Edition)  ---  1982-11-19
Coda (Remaster)  ---  1982-11-19
In Through the out Door (Deluxe Edition)  ---  1979-08-15
In Through the out Door (Remaster)  ---  1979-08-15
The Song Remains the Same (Remaster)  ---  1976-10-22
Presence (Deluxe Edition)  ---  1976-03-31
Presence (Remaster)  ---  1976-03-31
Physical Graffiti (Remaster)  ---  1975-02-24
Physical Graffiti (1994 Remaster)  ---  1975-02-24
Physical Graffiti (Deluxe Edition)  ---  1975-02-24
Houses of the Holy (Deluxe Edition)  ---  1973-03-28
Houses of the Holy (Remaster)  ---  1973-03-28
Led Zeppelin IV (Deluxe Edition)  ---  1971-11-08
Led Zeppelin IV (Remaster)  ---  1971-11-08
Led Zeppelin III (Deluxe Edition)  ---  1970-10-05
Led Zeppelin III (Deluxe Edition)  ---  1970-10-05
Led Zeppelin I

In [16]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums or int(album['release_date'][:4]) > 1983:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(album_name)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = r.json()['items']
    
    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'short_album_name': trim_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        data.append(f)

Coda (Deluxe Edition)
In Through the out Door (Deluxe Edition)
The Song Remains the Same (Remaster)
Presence (Deluxe Edition)
Physical Graffiti (Remaster)
Houses of the Holy (Deluxe Edition)
Led Zeppelin IV (Deluxe Edition)
Led Zeppelin III (Deluxe Edition)
Led Zeppelin II (Deluxe Edition)
Led Zeppelin (Deluxe Edition)


In [17]:
import pandas as pd

df = pd.DataFrame(data)

In [18]:
# convert release_date to an actual date, and sort by it
df['release_date'] = pd.to_datetime(df['release_date'])
df = df.sort_values(by='release_date')

# Zeppelin-specific: get rid of live album, remixes, vocal tracks, ...
df = df.query('short_album_name != "The Song Remains The Same"')
df = df[~df['track_name'].str.contains('Live|Mix|Track')]

In [19]:
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,short_album_name,release_date,album_id
143,0.445,0.703,9,-9.783,1,0.0859,0.0828,0.000003,0.0782,0.633,...,spotify:track:0FGhq1atMRDXBydpC4MlK2,https://api.spotify.com/v1/tracks/0FGhq1atMRDX...,https://api.spotify.com/v1/audio-analysis/0FGh...,166107,4,Good Times Bad Times - Remaster,Led Zeppelin (Deluxe Edition),Led Zeppelin,1969-01-12,5HNlYbQp7wKbKscWy7ceMp
144,0.411,0.466,4,-11.537,0,0.0332,0.3750,0.000085,0.1420,0.158,...,spotify:track:6bTvFs7doZSipBS7P7Epoo,https://api.spotify.com/v1/tracks/6bTvFs7doZSi...,https://api.spotify.com/v1/audio-analysis/6bTv...,402761,4,Babe I'm Gonna Leave You - Remaster,Led Zeppelin (Deluxe Edition),Led Zeppelin,1969-01-12,5HNlYbQp7wKbKscWy7ceMp
145,0.366,0.560,11,-10.361,0,0.0633,0.6470,0.007030,0.1800,0.361,...,spotify:track:0dF5j0jgcJBgMGGsT8drku,https://api.spotify.com/v1/tracks/0dF5j0jgcJBg...,https://api.spotify.com/v1/audio-analysis/0dF5...,388467,3,You Shook Me - Remaster,Led Zeppelin (Deluxe Edition),Led Zeppelin,1969-01-12,5HNlYbQp7wKbKscWy7ceMp
146,0.240,0.546,2,-10.026,1,0.0548,0.0898,0.006480,0.0898,0.429,...,spotify:track:68bhDgGmkopBrSHn9Ywstn,https://api.spotify.com/v1/tracks/68bhDgGmkopB...,https://api.spotify.com/v1/audio-analysis/68bh...,388667,4,Dazed and Confused - Remaster,Led Zeppelin (Deluxe Edition),Led Zeppelin,1969-01-12,5HNlYbQp7wKbKscWy7ceMp
147,0.328,0.575,7,-10.437,1,0.0439,0.3510,0.000556,0.6880,0.249,...,spotify:track:6xFvTvkm0Sw9iVI4TTuQvv,https://api.spotify.com/v1/tracks/6xFvTvkm0Sw9...,https://api.spotify.com/v1/audio-analysis/6xFv...,274694,4,Your Time Is Gonna Come - Remaster,Led Zeppelin (Deluxe Edition),Led Zeppelin,1969-01-12,5HNlYbQp7wKbKscWy7ceMp
